In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
%%html
<script>
    code_show=true; 
    function code_toggle() {
        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>

<img src="./qarnot_ligne.png" 
     width="30%" 
     align=right
     alt="Dask logo">
     
# Image builder


*Build your docker images without freezing your laptop!*

This notebook uses [Qarnot](https://computing.qarnot.com/en/) servers to build your docker image. It is based on the [kaniko project](https://github.com/GoogleContainerTools/kaniko).




In [3]:
import os
import ipywidgets as widgets
from tkinter import Tk, filedialog
import run_kaniko as qarnot_kaniko
from IPython.display import clear_output, display, HTML

## Add your Qarnot token

In [4]:
token = widgets.Password(
    placeholder='Qarnot token',
    description='* Qarnot token',
    disabled=False
)
display(token)

Password(description='* Qarnot token', placeholder='Qarnot token')

## Upload your Dockerfile to be built

In [5]:
file = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
display(file)

FileUpload(value={}, description='Upload')

## Upload your Docker context files
Select all files you want to add in the docker context

In [6]:
contextFiles = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)
display(contextFiles)

FileUpload(value={}, description='Upload', multiple=True)

## Select the Docker destination where to push the image
Format is {repo_name}/{image_name}:{tag}
Destination encompasses {repo_name}/{image_name}

In [7]:
repo = widgets.Text(description="* Repo destination",
                    placeholder="Destination docker repository",
                    width=200)
display(repo)

Text(value='', description='* Repo destination', placeholder='Destination docker repository')

In [8]:
tag = widgets.Text(description="Image tag",
                   placeholder="Destination docker tag",
                   width=200)
display(tag)

Text(value='', description='Image tag', placeholder='Destination docker tag')

## Select the Docker user and Docker pwd
They are used to push your image on your docker repository

In [9]:
user = widgets.Text(description="* Docker user",
                    placeholder="Docker user",
                    width=200)
display(user)

Text(value='', description='* Docker user', placeholder='Docker user')

In [10]:
pwd = widgets.Password(
    placeholder='Docker repository pwd',
    description='* Docker pwd',
    disabled=False
)
display(pwd)

Password(description='* Docker pwd', placeholder='Docker repository pwd')

## Launch the Qarnot computation

In [11]:
button = widgets.Button(description="Launch build!")
output = widgets.Output()
display(button, output)


def on_button_clicked(b):
    
    # Upload Dockerfile
    try:
        file_name = list(file.value.keys())[0]

        # Create folder input/
        ! mkdir -p input_binder

        # Write the Dockerfile in the binder container
        with open("./input_binder/Dockerfile", "wb") as fp:
            fp.write(file.value[file_name]['content'])
    except:
        print("An error occured with the Dockerfile upload")

    
    # Upload docker context files
    try:
        list_file_names = list(contextFiles.value.keys())

        # Write the context files in the binder container
        for f in list_file_names:
            with open("./input_binder/{}".format(f), "wb") as fp:
                fp.write(contextFiles.value[f]['content'])
    except:
        print("An error occured with the docker context files upload")
    
    # Launch computation
    try:
        # Set tag value to latest if not filled
        if tag.value == "":
            docker_tag = "latest"
        else:
            docker_tag = tag.value
    
        task_uuid = qarnot_kaniko.submit_task(token.value, repo.value, user.value, pwd.value, "{}/input_binder".format(os.getcwd()), docker_tag)

    except AttributeError:
        print("Some fields were not properly filled: check you selected a Dockerfile") 

        
button.on_click(on_button_clicked)

Button(description='Launch build!', style=ButtonStyle())

Output()

In [12]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')